scrab data from **6am - 11pm (about 11 hr)**, Mon-Fir

don't print the output! otherwise u can't open the file

In [1]:
import requests
import json
#from google.transit import gtfs_realtime_pb2
import urllib
import pandas as pd

1. acquire real-time rts data multiple times 

start from 6 am and end 11 pm (13 hr)

In [2]:
#getvehicles
# get route 1-11
#QxNvqGVVVdD4k3LgZjUgEEDz3
url1 = 'https://riderts.app/bustime/api/v3/getvehicles?key=RrQJYQvn2PpvPCRUVQepQvMnc&rt=1,2,3,5,6,7,8,9,10,11&tmres=s&format=json'
response1 = requests.get(url1)
data1 = str(response1.content)

# get route 12-26
url2="https://riderts.app/bustime/api/v3/getvehicles?key=RrQJYQvn2PpvPCRUVQepQvMnc&rt=12,13,15,16,17,20,23,24,25,26&tmres=s&format=json"
response2 = requests.get(url2)
data2 = str(response2.content)

# get route 27-119
url3="https://riderts.app/bustime/api/v3/getvehicles?key=RrQJYQvn2PpvPCRUVQepQvMnc&rt=27,33,34,35,37,38,43,46,75,119&tmres=s&format=json"
response3 = requests.get(url3)
data3 = str(response3.content)

# get route 122-711
url4="https://riderts.app/bustime/api/v3/getvehicles?key=RrQJYQvn2PpvPCRUVQepQvMnc&rt=122,121,76,78,125,126,127,150,711&tmres=s&format=json"
response4 = requests.get(url4)
data4 = str(response4.content)

url5="https://riderts.app/bustime/api/v3/getvehicles?key=RrQJYQvn2PpvPCRUVQepQvMnc&rt=11,1,12,27,122,118,76,78&tmres=s&format=json"
response5 = requests.get(url5)
data5 = str(response5.content)

url6="https://riderts.app/bustime/api/v3/getvehicles?key=RrQJYQvn2PpvPCRUVQepQvMnc&rt=11,21,28,36,40,121&tmres=s&format=json"
response6 = requests.get(url6)
data6 = str(response6.content)

2. renew the data every 15 sec, clean the empty values and merge them together

By default, one API key can make a maximum of **2400 (10hr)** requests per day.

In [3]:
#renew every 15 sec, check whether same as previous
import pprint
import time

nexttime = time.time()  # initializing

# switch to for i in range(2) to run finitely
# i=1
list6=[]

for i in range(1): #may set to 100000
    sesh = requests.Session()
    req1 = sesh.get(url1)
    list1=req1.json()['bustime-response']['vehicle']

    req2 = sesh.get(url2)
    list2=req2.json()['bustime-response']['vehicle']

    req3 = sesh.get(url3)
    list3=req3.json()['bustime-response']['vehicle']

    req6=sesh.get(url6)
    list8=req6.json()['bustime-response']['vehicle']
    
    req7=sesh.get(url5)
    list7=req7.json()['bustime-response']['vehicle']

    list5=list1+list2+list3+list7+list8 #实时更新
    if i==1:
        list6=list5
    else:
        list6=list6+list5
    #extract the data every 15 sec
    nexttime += 15
    sleeptime = nexttime - time.time()
    if sleeptime > 0:
        time.sleep(sleeptime)
 
        
        

In [4]:
# check if append all the historic data
len(list6)

64

In [9]:
df=pd.DataFrame(list6)
df.head(n=10)

,vid,tmstmp,lat,lon,hdg,pid,rt,des,pdist,dly,spd,tatripid,origtatripno,tablockid,zone,mode,psgld,stst,stsd
0,708,20230530 14:11:55,29.624938284432147,-82.37789609272588,300,476,1,Butler Plaza Transfer Station,24774,False,16,14149,795610,10011,,0,EMPTY,49620,2023-05-30
1,1203,20230530 14:11:54,29.6345837322791,-82.35604798850385,67,477,1,Rosa Parks/Downtown,13461,False,5,14184,795640,10012,,0,EMPTY,50280,2023-05-30
2,701,20230530 14:11:56,29.646045000000004,-82.32262,190,477,1,Rosa Parks/Downtown,29640,True,0,14183,795659,10013,,0,EMPTY,48660,2023-05-30
3,2305,20230530 14:11:57,29.627944104250787,-82.30581567405719,359,352,2,Walmart/Duval,19507,False,1,13966,795467,10021,,0,N/A,50400,2023-05-30
4,2102,20230530 14:11:52,29.651830056601902,-82.29392336971027,89,229,3,Rosa Parks/Downtown,18192,False,21,6880,794361,10031,,0,EMPTY,50400,2023-05-30
5,1105,20230530 14:11:53,29.652029004406835,-82.33071780973437,271,323,5,Oaks Mall,5131,False,0,12788,794704,10052,,0,EMPTY,50400,2023-05-30
6,1802,20230530 14:12:03,29.6577328130928,-82.40887113252678,0,323,5,Oaks Mall,32555,True,0,12787,794703,10051,,0,EMPTY,48960,2023-05-30
7,1810,20230530 14:12:07,29.64858768692644,-82.3223629780343,179,335,5,Rosa Parks/Downtown,30207,False,5,12824,794739,10053,,0,EMPTY,49740,2023-05-30
8,1805,20230530 14:11:54,29.645546,-82.322725,359,346,6,PineRidge/Walmart,0,False,0,13997,795470,10061,,0,EMPTY,50400,2023-05-30
9,2301,20230530 14:11:53,29.645802663974756,-82.30602096947408,356,349,7,Eastwood Meadows,6339,False,9,330,794136,10071,,0,N/A,50400,2023-05-30


In [6]:
df.tail(n=10)

,vid,tmstmp,lat,lon,hdg,pid,rt,des,pdist,dly,spd,tatripid,origtatripno,tablockid,zone,mode,psgld,stst,stsd
54,701,20230530 14:11:25,29.646045000000004,-82.32262,190,477,1,Rosa Parks/Downtown,29640,True,0,14183,795659,10013,,0,EMPTY,48660,2023-05-30
55,709,20230530 14:11:35,29.659220018852913,-82.2989831801922,270,44,11,Downtown/Health Dept,19263,False,27,6899,794377,10112,,0,EMPTY,50400,2023-05-30
56,2304,20230530 14:11:37,29.65946666666667,-82.30639333333332,36,104,11,Eastwood Meadows/Cedar Grove,10290,False,17,646,794164,10111,,0,N/A,50400,2023-05-30
57,2114,20230530 14:11:28,29.63375090209694,-82.35821632281133,153,500606,12,Reitz Union/UF Campus,15993,False,0,13110,795006,10122,,0,EMPTY,50520,2023-05-30
58,1016,20230530 14:11:32,29.645711628938667,-82.34621032961903,270,500606,12,Reitz Union/UF Campus,28160,False,16,13109,795005,10124,,0,EMPTY,49740,2023-05-30
59,1004,20230530 14:11:34,29.61874314323413,-82.37263808416066,174,500637,12,Butler Plaza Transfer Station,15356,False,33,13045,794941,10121,,0,EMPTY,50460,2023-05-30
60,1201,20230530 14:11:34,29.621428605429735,-82.38075149553131,326,500637,12,Butler Plaza Transfer Station,19000,False,0,13044,794940,10123,,0,EMPTY,49680,2023-05-30
61,1903,20230530 14:11:29,29.63853499945644,-82.34309291493675,346,500632,122,Animal Science NW 7 Ave/12 ST,5075,False,19,6857,794345,11221,,0,EMPTY,50640,2023-05-30
62,709,20230530 14:11:35,29.659220018852913,-82.2989831801922,270,44,11,Downtown/Health Dept,19263,False,27,6899,794377,10112,,0,EMPTY,50400,2023-05-30
63,2304,20230530 14:11:37,29.65946666666667,-82.30639333333332,36,104,11,Eastwood Meadows/Cedar Grove,10290,False,17,646,794164,10111,,0,N/A,50400,2023-05-30


In [7]:
#save to local 
from datetime import datetime
request_t = datetime.now()
import time
time_time = time.mktime(request_t.timetuple())
current_time=time.strftime('%Y-%m-%d-%I-%M',time.localtime(time_time))
current_time

import csv
import os
df.to_csv(current_time+".csv",index=False,encoding='utf-8')

In [8]:
#renew every 15 sec, check whether same as previous
import pprint
import time

nexttime = time.time()  # initializing

# switch to for i in range(2) to run finitely
# i=1
list6=[]

for i in range(360): #may set to 100000
    sesh = requests.Session()
    req1 = sesh.get(url1)
    list1=req1.json()['bustime-response']['vehicle']

    req2 = sesh.get(url2)
    list2=req2.json()['bustime-response']['vehicle']

    req3 = sesh.get(url3)
    list3=req3.json()['bustime-response']['vehicle']

    #req4 = sesh.get(url4)
    #list4=req4.json()['bustime-response']['vehicle']
    req6=sesh.get(url6)
    list8=req6.json()['bustime-response']['vehicle']
    
    req7=sesh.get(url5)
    list7=req7.json()['bustime-response']['vehicle']

    list5=list1+list2+list3+list7+list8 #实时更新
    if i==1:
        list6=list5
    else:
        list6=list6+list5
    #extract the data every 15 sec
    nexttime += 15
    sleeptime = nexttime - time.time()
    if sleeptime > 0:
        time.sleep(sleeptime)

KeyboardInterrupt: 

In [ ]:
#save to local 
from datetime import datetime
request_t = datetime.now()
import time
time_time = time.mktime(request_t.timetuple())
current_time=time.strftime('%Y-%m-%d-%I-%M',time.localtime(time_time))
current_time

import csv
import os
df.to_csv(current_time+".csv",index=False,encoding='utf-8')

: 

try with another API if the request got maximized:

question: as checked, 1, 12, 27, 122 can't extract

In [ ]:
#getvehicles
# get route 1-11
url1 = 'https://riderts.app/bustime/api/v3/getvehicles?key=QxNvqGVVVdD4k3LgZjUgEEDz3&rt=1,2,3,5,6,7,8,9,10,11&&tmres=s&format=json'
response1 = requests.get(url1)
data1 = str(response1.content)

# get route 12-26
url2="https://riderts.app/bustime/api/v3/getvehicles?key=QxNvqGVVVdD4k3LgZjUgEEDz3&rt=12,13,15,16,17,20,23,24,25,26&tmres=s&format=json"
response2 = requests.get(url2)
data2 = str(response2.content)

# get route 27-119
url3="https://riderts.app/bustime/api/v3/getvehicles?key=QxNvqGVVVdD4k3LgZjUgEEDz3&rt==27,33,34,35,37,38,43,46,75,119&tmres=s&format=json"
response3 = requests.get(url3)
data3 = str(response3.content)

# get route 122-711
url4="https://riderts.app/bustime/api/v3/getvehicles?key=QxNvqGVVVdD4k3LgZjUgEEDz3&rt=122,125,126,127,150,711&tmres=s&format=json"
response4 = requests.get(url4)
data4 = str(response4.content)

url5="https://riderts.app/bustime/api/v3/getvehicles?key=QxNvqGVVVdD4k3LgZjUgEEDz3&rt=11,1,12,27,122,711,150&tmres=s&format=json"
response5 = requests.get(url5)
data5 = str(response5.content)

url6="https://riderts.app/bustime/api/v3/getvehicles?key=QxNvqGVVVdD4k3LgZjUgEEDz3&rt=11,21,28,36,40,121&tmres=s&format=json"
response6 = requests.get(url6)
data6 = str(response6.content)

: 

: 

In [ ]:
nexttime = time.time()  # initializing

# switch to for i in range(2) to run finitely
# i=1
list6=[]

for i in range(1200): #max 5hr: 1200
    sesh = requests.Session()
    req1 = sesh.get(url1)
    list1=req1.json()['bustime-response']['vehicle']

    req2 = sesh.get(url2)
    list2=req2.json()['bustime-response']['vehicle']

    req3 = sesh.get(url3)
    list3=req3.json()['bustime-response']['vehicle']

   # req4 = sesh.get(url4)
  #  list4=req4.json()['bustime-response']['vehicle']
    req6=sesh.get(url6)
    list8=req6.json()['bustime-response']['vehicle']
    
    req7=sesh.get(url5)
    list7=req7.json()['bustime-response']['vehicle']

    list5=list1+list2+list3+list7+list8 #实时更新
    if i==1:
        list6=list5
    else:
        list6=list6+list5
    #extract the data every 15 sec
    nexttime += 15
    sleeptime = nexttime - time.time()
    if sleeptime > 0:
        time.sleep(sleeptime)
 

: 

In [ ]:
# check if append all the historic data
len(list6)

: 

In [ ]:
df=pd.DataFrame(list6)
df.head(n=10)

: 

In [ ]:
df.tail(n=10)

: 

In [ ]:
#save to local 
from datetime import datetime
request_t = datetime.now()
import time
time_time = time.mktime(request_t.timetuple())
current_time=time.strftime('%Y-%m-%d-%I-%M',time.localtime(time_time))
current_time

import csv
import os
df.to_csv(current_time+".csv",index=False,encoding='utf-8')

: 

In [ ]:
nexttime = time.time()  # initializing

# switch to for i in range(2) to run finitely
# i=1
list6=[]

for i in range(1240): #max 5hr: 1200
    sesh = requests.Session()
    req1 = sesh.get(url1)
    list1=req1.json()['bustime-response']['vehicle']

    req2 = sesh.get(url2)
    list2=req2.json()['bustime-response']['vehicle']

    req3 = sesh.get(url3)
    list3=req3.json()['bustime-response']['vehicle']

    req4 = sesh.get(url5)
    list4=req4.json()['bustime-response']['vehicle']
    
    req6=sesh.get(url6)
    
    list8=req6.json()['bustime-response']['vehicle']

    list5=list1+list2+list3+list4+list8 #实时更新
    if i==1:
        list6=list5
    else:
        list6=list6+list5
    #extract the data every 15 sec
    nexttime += 15
    sleeptime = nexttime - time.time()
    if sleeptime > 0:
        time.sleep(sleeptime)
        
        #save to local 
from datetime import datetime
request_t = datetime.now()
import time
time_time = time.mktime(request_t.timetuple())
current_time=time.strftime('%Y-%m-%d-%I-%M',time.localtime(time_time))
current_time

import csv
import os
df.to_csv(current_time+".csv",index=False,encoding='utf-8')

: 

In [ ]:
nexttime = time.time()  # initializing

# switch to for i in range(2) to run finitely
# i=1
list6=[]

for i in range(480): #max 5hr: 1200
    sesh = requests.Session()
    req1 = sesh.get(url1)
    list1=req1.json()['bustime-response']['vehicle']

    req2 = sesh.get(url2)
    list2=req2.json()['bustime-response']['vehicle']

    req3 = sesh.get(url3)
    list3=req3.json()['bustime-response']['vehicle']

    #req4 = sesh.get(url4)
    #list4=req4.json()['bustime-response']['vehicle']
    
    req7=sesh.get(url5)
    list7=req7.json()['bustime-response']['vehicle']

    list5=list1+list2+list3+list4+list7 #实时更新
    if i==1:
        list6=list5
    else:
        list6=list6+list5
    #extract the data every 15 sec
    nexttime += 15
    sleeptime = nexttime - time.time()
    if sleeptime > 0:
        time.sleep(sleeptime)
        
        #save to local 
from datetime import datetime
request_t = datetime.now()
import time
time_time = time.mktime(request_t.timetuple())
current_time=time.strftime('%Y-%m-%d-%I-%M',time.localtime(time_time))
current_time

import csv
import os
df.to_csv(current_time+".csv",index=False,encoding='utf-8')

: 

In [ ]:
nexttime = time.time()  # initializing

# switch to for i in range(2) to run finitely
# i=1
list6=[]

for i in range(240): #max 5hr: 1200
    sesh = requests.Session()
    req1 = sesh.get(url1)
    list1=req1.json()['bustime-response']['vehicle']

    req2 = sesh.get(url2)
    list2=req2.json()['bustime-response']['vehicle']

    req3 = sesh.get(url3)
    list3=req3.json()['bustime-response']['vehicle']

    req4 = sesh.get(url5)
    list4=req4.json()['bustime-response']['vehicle']

    list5=list1+list2+list3+list4 #实时更新
    if i==1:
        list6=list5
    else:
        list6=list6+list5
    #extract the data every 15 sec
    nexttime += 15
    sleeptime = nexttime - time.time()
    if sleeptime > 0:
        time.sleep(sleeptime)
        
        #save to local 
from datetime import datetime
request_t = datetime.now()
import time
time_time = time.mktime(request_t.timetuple())
current_time=time.strftime('%Y-%m-%d-%I-%M',time.localtime(time_time))
current_time

import csv
import os
df.to_csv(current_time+".csv",index=False,encoding='utf-8')

: 

In [ ]:
url5="https://riderts.app/bustime/api/v3/getvehicles?key=RrQJYQvn2PpvPCRUVQepQvMnc&rt=118,121&tmres=s&format=json"
response5 = requests.get(url5)
data5 = str(response5.content)

: 

In [ ]:
import time
nexttime = time.time()  # initializing

# switch to for i in range(2) to run finitely
# i=1
list6=[]

for i in range(1200): #max 5hr: 1200
    sesh = requests.Session()
    req5 = sesh.get(url5)
    list5=req5.json()['bustime-response']['vehicle']

    if i==1:
        list6=list5
    else:
        list6=list6+list5
    #extract the data every 15 sec
    nexttime += 15
    sleeptime = nexttime - time.time()
    if sleeptime > 0:
        time.sleep(sleeptime)
        
        #save to local 
from datetime import datetime
request_t = datetime.now()
import time
time_time = time.mktime(request_t.timetuple())
current_time=time.strftime('%Y-%m-%d-%I-%M',time.localtime(time_time))
current_time

import csv
import os
df.to_csv(current_time+".csv",index=False,encoding='utf-8')

: 

In [ ]:
list6

: 

In [ ]:
df=pd.DataFrame(list6)

: 

In [ ]:
df

: 

In [ ]:
from datetime import datetime
request_t = datetime.now()
import time
time_time = time.mktime(request_t.timetuple())
current_time=time.strftime('%Y-%m-%d-%I-%M',time.localtime(time_time))
df.to_csv(current_time+".csv",index=False,encoding='utf-8')

: 

: 